In [1]:
import os
import sys
import re
import pandas as pd
import numpy as np
from enum import Enum
import zipfile
from typing import Optional, Union, List, Callable, Dict, Tuple
from dataclasses import dataclass, make_dataclass, field
from abc import ABC

PKG_ROOT = os.path.dirname(os.path.realpath(os.getcwd()))
if not PKG_ROOT in sys.path:
    sys.path.append(PKG_ROOT)

In [2]:
class IDS:
    
    @classmethod
    def make_names_old(cls, name:str, df:pd.DataFrame):
        '''
        Construct dataclass or nested dataclass from a dataframe
        '''
        if df.shape[1] == 2:
            data = [(_[1], str, field(default=_[0])) for _ in df.values]
            return make_dataclass(name, data)
        else:
            group = df.groupby(df.columns[0])
            data = [(k, dataclass, field(default=cls.make_names_old(k, v.drop(v.columns[0], axis=1)))) for k, v in group]
            return make_dataclass(name, data)

    @classmethod
    def make_names_(cls, name:str, df:pd.DataFrame):
        '''
        Construct dataclass or nested dataclass from a dataframe
        '''
        if df.shape[1] == 2:
            data = [(_[1], str, field(default=_[0])) for _ in df.values]
            return make_dataclass(name, data)
        else:
            group = df.groupby(df.columns[0])
            if group.ngroups == 1:
                return cls.make_names_(name, df.drop(df.columns[0], axis=1))
            data = [(k, dataclass, field(default=cls.make_names_(k, v.drop(v.columns[0], axis=1)))) for k, v in group]
            return make_dataclass(name, data)
        
    @classmethod
    def make_names(cls, name:str, df:pd.DataFrame, ignoreCols:List=[]):
        return cls.make_names_(name=name, df=df[[_ for _ in df.columns if not _ in ignoreCols]])

    @staticmethod
    def make_enum(name:str, df:pd.DataFrame, nameCol:int=1, valueCol:Optional[Union[int, list]]=None):
        if df.shape[1] == 1:
            df = df.reset_index()
        nameCol = df.columns[nameCol]
        if valueCol is None:
            valueCol = df.columns.difference([nameCol])
        else:
            valueCol = df.columns[valueCol]
        if len(valueCol) == 1:
            valueCol = valueCol[0]
        if isinstance(valueCol, (list, tuple, pd.Index)):
            return Enum(name, df.set_index(nameCol)[valueCol].apply(tuple, axis=1).to_dict())
        else:
            return Enum(name, df.set_index(nameCol)[valueCol].to_dict())
        
    @staticmethod
    def make_map(df, cols:List):
        d1 = df[df.columns[cols[:2]]].set_index(df.columns[cols[0]]).drop_duplicates().to_dict()
        d2 = df[df.columns[cols[:2]]].set_index(df.columns[cols[1]]).drop_duplicates().to_dict()
        d1.update(d2)
        return d1
    
    @classmethod
    def clean_duplicates(cls, df:pd.DataFrame, valueCol:int=-1, groupby:Optional[Union[List, str]]=None):
        if groupby is None:
            col = df.columns[valueCol]
            df = df.groupby(col).apply(lambda x: x.reset_index())
            if df.index.nlevels > 1:
                df = df.droplevel(0)
            df[col] = df.apply(lambda x: f'{re.sub("_+", "_", x[col])}{("_" + str(x.name)) if x.name > 0 else ""}', axis=1)
            return df.reset_index(drop=True).drop('index', axis=1)
        else:
            return df.groupby(groupby).apply(lambda x:cls.clean_duplicates(x, valueCol=valueCol)).reset_index(drop=True)

In [3]:
class BlockBase:
    
    @staticmethod
    def bytes2Num(barray:bytes, signed:bool=False, order:str='little'):
        return int.from_bytes(barray, signed=signed, byteorder=order)
    
    @staticmethod
    def num2Bytes(num:int, length:int, signed=False, order='little'):
        return num.to_bytes(length, signed=signed, byteorder=order)
    
    @staticmethod
    def bytes2Str(barray:bytes):
        ba = np.array(bytearray(barray))
        p = np.argmax(np.logical_or(ba > 127
                                   ,ba < 32))
        if p == 0: ## All strings will have >1 length so when p == 0, it only means the string is the whole length of the bytearray
            p = len(barray)
        return barray[:p].decode('latin')
    
    @staticmethod
    def to_int(num:float):
        '''
        Integer will be coerced to float if np.nan exists in the same column
        Force the value to int, if np.nan then return np.nan
        '''
        if isinstance(num, str): #0x0000 in dataframe
            return int(num, 16)
        if num == None:
            return None
        if np.isnan(num):
            return None
        return int(num)


In [4]:
class ValueBlock(BlockBase):
    
    def __init__(self
                 ,pbuffer: bytearray  #contents of parent segment
                 ,name: str
                 ,valueLoc: int
                 ,size: int
                 ,signed: bool = False
                 ,order: str = 'little'
                 ,optimize: Optional[int] = None
                 ,skiphead: int = 0
                 ,skiptail: int = 0):
        self.pbuffer = pbuffer
        self.name = name
        self.valueLoc = self.to_int(valueLoc)
        self.size = self.to_int(size)
        self.signed = signed
        self.order = order
        self.optimize = self.to_int(optimize)
        self.skiphead = skiphead
        self.skiptail = skiptail
        self.notVoid = self.not_void()
    
    def not_void(self):
        if any([pd.isnull(self.valueLoc), pd.isnull(self.size)]):
            return False
        if any([self.pbuffer is None
                ,len(self.pbuffer)==0
                ,self.valueLoc < 0
                ,self.size <= 0
                ,self.valueLoc + self.size > len(self.pbuffer)]):
            return False
        return True
    
    def bind_buffer(self, buffer:bytearray):
        self.pbuffer = buffer
        return self
     
    @property
    def bytes(self):
        if self.not_void:
            return self.pbuffer[self.valueLoc+self.skiphead
                               :self.valueLoc+self.size+self.skiphead-self.skiptail]
        return None
        
    @property
    def value(self):
        if self.not_void:
            return self.bytes2Num(self.bytes, self.signed, self.order)
        return None
    
    @property
    def string(self):
        if self.not_void:
            return self.bytes2Str(self.bytes)
        return None
    
    def set_value(self, value:Union[int, Enum]):            
        if hasattr(value, 'value'):
                value = value.value
        value = self.to_int(value)
        if self.notVoid and value is not None:
            self.pbuffer[self.valueLoc+self.skiphead
                         :self.valueLoc+self.size+self.skiphead-self.skiptail] = self.num2Bytes(value, self.size, self.signed, self.order) 
    
            
    def inc_value(self, inc_value:int=1):
        if self.notVoid:
            self.set_value(self.value + inc_value)
    